# GeoJSON - Find Ploygon for Cordinates


**Portfolio Project**

Finding if a pair of cordinateds are witin a GeoJSON polygon and if so, which one are they in.

This information was used to connect geospatial information to a Power BI Dashboard.

In [ ]:
# Import the all libraries that are used:
import pandas as pd
import json
from shapely.geometry import shape, Point
from google.colab import files, drive

In [ ]:
# Upload your GeoJSON file containing polygons
uploaded_geojson = files.upload()
geojson_filename = next(iter(uploaded_geojson))

Saving Sepa colourful geo json(simplified geojson).geojson to Sepa colourful geo json(simplified geojson).geojson


In [ ]:
# Read the GeoJSON file
with open(geojson_filename) as f:
    data = json.load(f)

In [ ]:
# Create a list to store polygon names and geometries
polygons = []

In [ ]:
# Extract polygon names and geometries from features
for feature in data['features']:
    name = feature['properties']['CATCHMENT']
    geometry = feature.get('geometry')  # Handle None value
    if geometry:
        polygons.append({'name': name, 'geometry': shape(geometry)})

# Rest of your code remains unchanged...
# (Iterate over coordinates, check if point is within polygons, etc.)

In [ ]:
# Upload your list of coordinates (CSV or Excel file)
uploaded_coordinates = files.upload()
coordinates_filename = next(iter(uploaded_coordinates))

Saving Projects_Sites_Joined.csv to Projects_Sites_Joined.csv


In [ ]:
# Read the coordinates file into a dataframe
df = pd.read_csv(coordinates_filename)  # You can adjust this line based on your file format

In [ ]:
# Create empty list to store results
results = []

In [ ]:
# Iterate over the coordinates in the dataframe
for index, row in df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    point = Point(longitude, latitude)

    # Check if the point is within any of the polygons
    is_within_polygon = False
    polygon_name = None

    for polygon in polygons:
        # Apply ST_MakeValid() to the polygon geometry
        valid_polygon = polygon['geometry'].buffer(0)  # Buffer with zero distance to fix invalidity
        if valid_polygon.contains(point):
            is_within_polygon = True
            polygon_name = polygon['name']
            break

    # Append the result to the list
    results.append({'Latitude': latitude, 'Longitude': longitude, 'IsWithinPolygon': is_within_polygon, 'PolygonName': polygon_name})


In [ ]:
# Create a new dataframe with the results
results_df = pd.DataFrame(results)

In [ ]:
display(results_df)


,Latitude,Longitude,IsWithinPolygon,PolygonName
0,57.506681,-1.842433,True,31
1,55.800896,-3.110410,True,63
2,55.800896,-3.110410,True,63
3,57.090259,-3.967432,True,28
4,55.503638,-4.341328,True,89
...,...,...,...,...
212,59.531858,-1.630355,True,1496
213,55.866022,-4.337900,True,93
214,55.837328,-4.218698,True,93
215,56.382744,-3.383700,True,45


In [ ]:
# Merge the original dataframe with the results dataframe
merged_df = pd.merge(df, results_df, on=['Latitude', 'Longitude'], how='left')

# Display the merged dataframe
display(merged_df)

,Project_ID,Period,Project_Key,Project_Name,Primary_Local_Authority,Longitude,Latitude,Postcode,Equipment_Number,IsWithinPolygon,PolygonName
0,532714,2023_24_P09,2023_24_P09:5327140000,Forehill DSR Repairs,Fife,-1.842433,57.506681,AB42 3JY,5000063767,True,31
1,532714,2023_24_P09,2023_24_P09:5327140000,Forehill DSR Repairs,Fife,-1.842433,57.506681,AB42 3JY,5000063767,True,31
2,502246,2023_24_P09,2023_24_P09:5022460000,Rosebery WTW Lime to Caustic,Midlothian,-3.110410,55.800896,EH23 4SQ,5000004128,True,63
3,502246,2023_24_P09,2023_24_P09:5022460000,Rosebery WTW Lime to Caustic,Midlothian,-3.110410,55.800896,EH23 4SQ,5000004128,True,63
4,501390,2023_24_P09,2023_24_P09:5013900000,WTW000627 - ROSEBERY WTW 1940 NT305570,Midlothian,-3.110410,55.800896,EH23 4SQ,5000004128,True,63
...,...,...,...,...,...,...,...,...,...,...,...
278,503269,2023_24_P09,2023_24_P09:5032690000,Dalmarnock WWTW Improvements,Glasgow City,-4.218698,55.837328,G40 4HU,5000004737,True,93
279,508752,2023_24_P09,2023_24_P09:5087520000,E-WW-NI-150920-PERTH CITY WWTW-IFAS,Perth and Kinross,-3.383700,56.382744,PH2 8DF,5000158942,True,45
280,508752,2023_24_P09,2023_24_P09:5087520000,E-WW-NI-150920-PERTH CITY WWTW-IFAS,Perth and Kinross,-3.383700,56.382744,PH2 8DF,5000158942,True,45
281,508752,2023_24_P09,2023_24_P09:5087520000,E-WW-NI-150920-PERTH CITY WWTW-IFAS,Perth and Kinross,-3.383498,56.383411,PH2 8QJ,5000062451,True,45


In [ ]:
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Save the results to a CSV file
merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/output.csv', index=False)

# Provide a link to download the CSV file
print("Results saved to output.csv. You can download it using the link below:")
print(f"Download CSV")


Results saved to output.csv. You can download it using the link below:
Download CSV
